<a href="https://colab.research.google.com/github/Sudeeppp-Mishra/LeafLens/blob/main/LeafLens_Train_on_DataSets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LeafLens: AI-Based Plant Disease Detection

LeafLens classifies plant leaf images into healthy or diseased categories.
We use a pretrained ResNet18 model (transfer learning) for accuracy and faster training.

**Libraries used:**
- PIL / OpenCV / NumPy: Image loading and preprocessing
- PyTorch / Torchvision: Model training and evaluation
- Matplotlib: Visualization of metrics
- PySide6 / pyttsx3 / gTTS: GUI + audio output (later integration)

**Dataset:** PlantVillage (Image Classification)  
**Platform:** Google Colab (GPU)

## Why GPU?

Deep learning models require heavy computation.  
Using GPU in Colab speeds up training significantly.

## Mount Google Drive

Accessing PlantVillage dataset stored in Google Drive.  
This avoids repeated uploads and keeps large datasets organized.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Required Libraries

Purpose

We import all required libraries for:

	•	Deep learning (PyTorch)
	•	Image processing (OpenCV, PIL)
	•	Dataset handling
	•	Visualization

We use:

	•	OpenCV → fast, robust image reading
	•	PIL → transformations compatibility
	•	PyTorch → CNN training framework

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models
from sklearn.model_selection import train_test_split

## Check GPU

We confirm if GPU is available.  
Training on GPU is much faster than CPU.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## Dataset Path

Why?

We tell Python where PlantVillage dataset is located.

In [ ]:
DATASET_DIR = "/content/drive/MyDrive/datasets/PlantVillage"
print("Classes found:", os.listdir(DATASET_DIR))

Classes found: ['Tomato_healthy', 'PlantVillage', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato_Leaf_Mold', 'Potato___Early_blight', 'Tomato_Septoria_leaf_spot', 'Potato___Late_blight', 'Tomato_Early_blight', 'Tomato__Target_Spot', 'Pepper__bell___healthy', 'Tomato__Tomato_mosaic_virus', 'Tomato_Bacterial_spot', 'Potato___healthy', 'Pepper__bell___Bacterial_spot', 'Tomato_Late_blight']


## Load Image Paths and Labels

Why??

CNN does not understand folders.

We convert:

	•	Folder names → class labels
	•	Images → file paths

In [ ]:
image_paths = []
labels = []
class_names = sorted(os.listdir(DATASET_DIR))

class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

for cls in class_names:
    cls_path = os.path.join(DATASET_DIR, cls)
    if not os.path.isdir(cls_path):
        continue
    for img in os.listdir(cls_path):
        image_paths.append(os.path.join(cls_path, img))
        labels.append(class_to_idx[cls])

print("Total images:", len(image_paths))
print("Total classes:", len(class_names))

Total images: 20654
Total classes: 16


## Training-Validation Split (80-20)

WHY THIS IS MANDATORY

Why not train on 100% data?

Because:
	•	Model may memorize images (overfitting)
	•	You cannot measure real performance

80% → learning

20% → evaluation (unseen images)

In [ ]:
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42, stratify=labels
)

print("Training images:", len(train_paths))
print("Validation images:", len(val_paths))

Training images: 16523
Validation images: 4131


## Image Preprocessing (OpenCV + PIL)

Why preprocessing BEFORE training

CNN expects:

	•	Same image size
	•	Normalized pixel values
	•	Clean data

We use:

	•	OpenCV → read image
	•	PIL → apply transforms
	•	Normalization → faster convergence


In [ ]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

## SAFE Dataset Class

WHY THIS IS CRITICAL

	•	Some images may be corrupted
	•	OpenCV returns None
	•	This prevents infinite freeze

In [ ]:
class SafePlantDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]

        try:
            image = cv2.imread(path)
            if image is None:
                raise ValueError("Corrupted image")

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            if self.transform:
                image = self.transform(image)

            return image, self.labels[idx]

        except:
            return self.__getitem__((idx + 1) % len(self.paths))

## DataLoader

- Loads batches efficiently  
- Shuffles training data for better convergence

In [ ]:
train_dataset = SafePlantDataset(train_paths, train_labels, train_transform)
val_dataset = SafePlantDataset(val_paths, val_labels, val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

## Model Selection - EfficientNet
Why EfficientNet?

	•	Better accuracy than ResNet
	•	Fewer parameters
	•	Faster on limited GPU

In [ ]:
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(class_names))
model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 227MB/s]


## Loss Function & Optimizer

- CrossEntropyLoss for multi-class classification  
- Adam optimizer adapts learning rate for faster convergence

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

## Model Training

- Forward pass → Compute loss → Backpropagate → Update weights  
- Track training and validation accuracy  
- GPU is used if available for faster computation

Installing tqdm to see progress of training

In [ ]:
!pip install tqdm

In [ ]:
import torch
from tqdm import tqdm
import os

# Parameters
epochs = 5
checkpoint_dir = "/content/drive/MyDrive/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
save_every_n_batches = 100  # adjust for speed vs safety

# OPTIONAL: resume from checkpoint
start_epoch = 0
start_batch = 0
checkpoint_path = "/content/drive/MyDrive/checkpoints/checkpoint_epoch5_end.pth" # path of last checkpoint

if checkpoint_path and os.path.exists(checkpoint_path):
    print(f"Resuming training from checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    start_batch = checkpoint['batch'] + 1
    print(f"Resuming from epoch {start_epoch+1}, batch {start_batch}")

# Training loop
for epoch in range(start_epoch, epochs):
    model.train()
    running_loss = 0.0

    print(f"\nEpoch {epoch+1}/{epochs}")

    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training", leave=False)

    for i, (images, labels) in progress_bar:
        # Skip batches if resuming
        if epoch == start_epoch and i < start_batch:
            continue

        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

        # Save checkpoint every N batches
        if (i + 1) % save_every_n_batches == 0:
            checkpoint_file = os.path.join(checkpoint_dir, f"checkpoint_epoch{epoch+1}_batch{i+1}.pth")
            torch.save({
                'epoch': epoch,
                'batch': i,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            }, checkpoint_file)
            print(f"\nCheckpoint saved: {checkpoint_file}")

    avg_loss = running_loss / len(train_loader)
    print(f"Training Loss Epoch {epoch+1}: {avg_loss:.4f}")

    # Optional: save checkpoint at end of each epoch
    epoch_checkpoint_file = os.path.join(checkpoint_dir, f"checkpoint_epoch{epoch+1}_end.pth")
    torch.save({
        'epoch': epoch,
        'batch': i,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, epoch_checkpoint_file)
    print(f"Epoch {epoch+1} checkpoint saved: {epoch_checkpoint_file}")

Resuming training from checkpoint: /content/drive/MyDrive/checkpoints/checkpoint_epoch5_end.pth
Resuming from epoch 5, batch 517

Epoch 5/5


Training Loss Epoch 5: 0.0000
Epoch 5 checkpoint saved: /content/drive/MyDrive/checkpoints/checkpoint_epoch5_end.pth


## Saving the trained final model

In [ ]:
# Path where final model will be saved
final_model_path = "/content/drive/MyDrive/leaflens_final_model.pth"

# Save only the trained model weights (recommended)
torch.save(model.state_dict(), final_model_path)

print("Final trained model saved at:", final_model_path)

##Loading the Trained LeafLens Model

This cell rebuilds the EfficientNet-B0 model architecture and loads the previously saved trained weights from Google Drive. The final classification layer is configured for 16 plant disease classes. After loading the trained parameters, the model is moved to the available GPU and switched to evaluation mode. This prepares the model for testing and making predictions without further training.

In [ ]:
model = models.efficientnet_b0(pretrained=False)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 16)
model.load_state_dict(torch.load("/content/drive/MyDrive/leaflens_final_model.pth", map_location=device))
model = model.to(device)
model.eval()


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

## Confirming GPU + eval mode

In [ ]:
print(next(model.parameters()).is_cuda)  # Should print True
print(model.training)                     # Should print False


True
False


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np
import torch

model.eval()  # Ensure evaluation mode

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Accuracy
val_accuracy = accuracy_score(all_labels, all_preds)
print(f"\nValidation Accuracy: {val_accuracy*100:.2f}%")

# Classification Report (Precision, Recall, F1-score)
print("\nClassification Report:\n")
print(classification_report(all_labels, all_preds, target_names=class_names))

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
print("\nConfusion Matrix:\n")
print(cm)


Validation Accuracy: 3.80%

Classification Report:

                                             precision    recall  f1-score   support

              Pepper__bell___Bacterial_spot       0.03      0.15      0.05       199
                     Pepper__bell___healthy       0.08      0.03      0.05       296
                               PlantVillage       0.00      0.00      0.00         0
                      Potato___Early_blight       0.00      0.00      0.00       200
                       Potato___Late_blight       0.00      0.00      0.00       201
                           Potato___healthy       0.00      0.07      0.00        30
                      Tomato_Bacterial_spot       0.08      0.03      0.05       426
                        Tomato_Early_blight       0.00      0.00      0.00       200
                         Tomato_Late_blight       0.08      0.16      0.11       382
                           Tomato_Leaf_Mold       0.01      0.01      0.01       190
           

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
